In [ ]:
!nvidia-smi

Fri Jul 15 12:43:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

In [ ]:
# Import required packages
import sys, os, re, codecs
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
import keras
import gc
import gensim
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.graph_objects as go

from tqdm.notebook import tqdm
import plotly.graph_objects as go
import re
import json
import copy
import collections

import hazm
from __future__ import unicode_literals
from hazm import *

In [ ]:
data  = pd.read_excel('/content/drive/MyDrive/Data-mining/Labeled-Sentences - 5.xlsx')
#data = data[['text','sentiment']]
print()

In [ ]:
data.columns

Index(['text', 'sentiment', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
data=data.drop('Unnamed: 2',axis=1)
data=data.drop('Unnamed: 3',axis=1)
data=data.drop('Unnamed: 4',axis=1)


In [ ]:
data.head()

,text,sentiment
0,سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...,2
1,سلام بنده با همراه خانوادم چهار روز در این هتل...,2
2,ما از تاریخ ۵اسفند 1400تا۸اسفند 1400 در این هت...,2
3,من 2روز در اين هتل به همراه همسرم اقامت داشتم ...,2
4,سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...,2


In [ ]:
data['text']

0       سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...
1       سلام بنده با همراه خانوادم چهار روز در این هتل...
2       ما از تاریخ ۵اسفند 1400تا۸اسفند 1400 در این هت...
3       من 2روز در اين هتل به همراه همسرم اقامت داشتم ...
4       سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...
                              ...                        
1628        هتل بسيار تميز و داراي استانداردهاي روز دنيا 
1629                                   طراحي مدرن و عالي 
1630                   پرسنل آموزش ديده جوان و تحصيلكرده 
1631                             اتاقهاي فوق العاده تميز 
1632               . من واقعا أقامت آرامي را تجربه كردم .
Name: text, Length: 1633, dtype: object

In [ ]:
data.shape

(1633, 2)

In [ ]:
print(data[:100])

                                                 text  sentiment
0   سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...          2
1   سلام بنده با همراه خانوادم چهار روز در این هتل...          2
2   ما از تاریخ ۵اسفند 1400تا۸اسفند 1400 در این هت...          2
3   من 2روز در اين هتل به همراه همسرم اقامت داشتم ...          2
4   سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...          2
..                                                ...        ...
95  باسلام من وخانواده ابان 1400 چند روز در این هت...          2
96  به نظر من یکی از هتلهای ۵ستاره واقعی ایران است...          2
97  با سلام هتل هلیا پذیرش خوب - به حرم نزدیک - فض...          0
98               واقعا عالی هتل تمیز رفتار پرسنل عالی          2
99  سلام.من سه روز در هتل اقامت داشتم و از نظر نزد...          2

[100 rows x 2 columns]


In [ ]:
data.columns = ['comment', 'rate']

In [ ]:
# print data information
print('data information')
print(data.info(), '\n')

# print missing values information
print('missing values stats')
print(data.isnull().sum(), '\n')

# print some missing values
print('some missing values')
print(data[data['rate'].isnull()].iloc[:5], '\n')

data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1633 entries, 0 to 1632
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1632 non-null   object
 1   rate     1633 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.6+ KB
None 

missing values stats
comment    1
rate       0
dtype: int64 

some missing values
Empty DataFrame
Columns: [comment, rate]
Index: [] 



In [ ]:
# handle some conflicts with the dataset structure
# you can find a reliable solution, for the sake of the simplicity
# I just remove these bad combinations!
data['rate'] = data['rate'].apply(lambda r: r if r < 6 else None)

data = data.dropna(subset=['rate'])
data = data.dropna(subset=['comment'])
data = data.drop_duplicates(subset=['comment'], keep='first')
data = data.reset_index(drop=True)


# previous information after solving the conflicts

# print data information
print('data information')
print(data.info(), '\n')

# print missing values information
print('missing values stats')
print(data.isnull().sum(), '\n')

# print some missing values
print('some missing values')
print(data[data['rate'].isnull()].iloc[:5], '\n')

data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1604 entries, 0 to 1603
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1604 non-null   object
 1   rate     1604 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.2+ KB
None 

missing values stats
comment    0
rate       0
dtype: int64 

some missing values
Empty DataFrame
Columns: [comment, rate]
Index: [] 



In [ ]:
# calculate the length of comments based on their words
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))

In [ ]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 1199


In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(data, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 96.01% of the whole!


In [ ]:
minlim, maxlim = 3, 256

In [ ]:
# remove comments with the length of fewer than three words
data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
unique_rates = list(sorted(data['rate'].unique()))
print(f'We have #{len(unique_rates)}: {unique_rates}')

We have #3: [0, 1, 2]


In [ ]:
fig = go.Figure()

groupby_rate = data.groupby('rate')['rate'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of rate within comments',
    xaxis_title_text='Rate',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
len(data)

1540

In [ ]:
def rate_to_label(rate):
    if rate == 0:
        return 'negative'
    elif rate == 1 :
        return 'neural'
    else:
        return 'positive'


data['label'] = data['rate'].apply(lambda t: rate_to_label(t))
labels = list(sorted(data['label'].unique()))
data.head()

,comment,rate,comment_len_by_words,label
0,سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...,2,58.0,positive
1,سلام بنده با همراه خانوادم چهار روز در این هتل...,2,63.0,positive
2,ما از تاریخ ۵اسفند 1400تا۸اسفند 1400 در این هت...,2,51.0,positive
3,من 2روز در اين هتل به همراه همسرم اقامت داشتم ...,2,24.0,positive
4,سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...,2,23.0,positive


In [ ]:
data['rate']

0       2
1       2
2       2
3       2
4       2
       ..
1535    2
1536    2
1537    2
1538    2
1539    2
Name: rate, Length: 1540, dtype: int64

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
# cleaning comments
data['cleaned_comment'] = data['comment'].apply(cleaning)


# calculate the length of comments based on their words
data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)

data.head()

,comment,rate,comment_len_by_words,label,cleaned_comment,cleaned_comment_len_by_words
0,سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...,2,58.0,positive,سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...,54
1,سلام بنده با همراه خانوادم چهار روز در این هتل...,2,63.0,positive,سلام بنده با همراه خانوادم چهار روز در این هتل...,63
2,ما از تاریخ ۵اسفند 1400تا۸اسفند 1400 در این هت...,2,51.0,positive,ما از تاریخ ۵اسفند ۱۴۰۰تا۸اسفند ۱۴۰۰ در این هت...,51
3,من 2روز در اين هتل به همراه همسرم اقامت داشتم ...,2,24.0,positive,من ۲روز در این هتل به همراه همسرم اقامت داشتم ...,24
4,سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...,2,23.0,positive,سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...,23


In [ ]:
data = data[['cleaned_comment', 'label']]
data.columns = ['comment', 'label']
data.head()

,comment,label
0,سلام من یک ماه پیش در هتل شمس قشم بودم واقعا ه...,positive
1,سلام بنده با همراه خانوادم چهار روز در این هتل...,positive
2,ما از تاریخ ۵اسفند ۱۴۰۰تا۸اسفند ۱۴۰۰ در این هت...,positive
3,من ۲روز در این هتل به همراه همسرم اقامت داشتم ...,positive
4,سفر بسیار خوبی بود هتل بسیار زیبا و قشنگی بود ...,positive


In [ ]:
print(f'We have #{len(labels)} labels: {labels}')

We have #3 labels: ['negative', 'neural', 'positive']


In [ ]:
fig = go.Figure()

groupby_label = data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
negative_data = data[data['label'] == 'negative']
neural_data = data[data['label'] == 'neural']
positive_data = data[data['label'] == 'positive']

cutting_point = min(len(negative_data), len(neural_data) ,len(positive_data) )

if cutting_point <= len(negative_data):
    negative_data = negative_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(neural_data):
    neural_data = neural_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(positive_data):
    positive_data = positive_data.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([negative_data, neural_data, positive_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489 entries, 0 to 488
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  489 non-null    object
 1   label    489 non-null    object
dtypes: object(2)
memory usage: 7.8+ KB


In [ ]:
fig = go.Figure()

groupby_label = new_data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
new_data['label_id'] = new_data['label'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(396, 3)
(44, 3)
(49, 3)


In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'ab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
# create a key finder based on label 2 id and id to label

label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'negative': 0, 'neural': 1, 'positive': 2}
id2label: {0: 'negative', 1: 'neural', 2: 'positive'}


In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "neural",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "neural": 1,
    "positive": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [ ]:
idx = np.random.randint(0, len(train))
sample_comment = train.iloc[idx]['comment']
sample_label = train.iloc[idx]['label']

print(f'Sample: \n{sample_comment}\n{sample_label}')

Sample: 
بدترین تجربه اقامت درهتل به اصطلاح ۵ستاره را داشتم به نظرم اگه میخوایدپولتونوبریزید دوربریدبه هتلی مثل هتل داریوش که از بدو ورود ضعف مدیریت درتمام قسمتهاش به چشم میخوره. اطلاعات سایتشون که دروغه مثلا نوشته شده استخرجهت بانوان که سالهاست راه اندازی نشده یاسرویس کافی شاپ که فقط ۲مورد نوشیدنی گرم در ظروف نامناسب و لب پر شده سرومیکنن. کیفیت سرویس دهی رستورانهاشون خیلی ضعیفه مثلا بایدساعتها بایستی تاظرف خالی یه نوع غذاپرشه. خلاصه فکرکنم آقای ثابت هتل روبیخیال شدن واونومیخوان حراج کنن که همچین مدیرانی رواونجاسرکارگذاشتن. فقط ورودی هتل زیباست ولی ازبعدش همه چی خرابه. سالن ورزشی هتل ۵ستاره تمام دستگاهاش فرسوده وخرابه. هتل فاقد کافی نته. به نظرم اینجاخدماتش بایه مسافرخونه فرق چندانی نداره فقط ظاهرشوتغییردادن
negative


In [ ]:
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: بدترین تجربه اقامت درهتل به اصطلاح ۵ستاره را داشتم به نظرم اگه میخوایدپولتونوبریزید دوربریدبه هتلی مثل هتل داریوش که از بدو ورود ضعف مدیریت درتمام قسمتهاش به چشم میخوره. اطلاعات سایتشون که دروغه مثلا نوشته شده استخرجهت بانوان که سالهاست راه اندازی نشده یاسرویس کافی شاپ که فقط ۲مورد نوشیدنی گرم در ظروف نامناسب و لب پر شده سرومیکنن. کیفیت سرویس دهی رستورانهاشون خیلی ضعیفه مثلا بایدساعتها بایستی تاظرف خالی یه نوع غذاپرشه. خلاصه فکرکنم آقای ثابت هتل روبیخیال شدن واونومیخوان حراج کنن که همچین مدیرانی رواونجاسرکارگذاشتن. فقط ورودی هتل زیباست ولی ازبعدش همه چی خرابه. سالن ورزشی هتل ۵ستاره تمام دستگاهاش فرسوده وخرابه. هتل فاقد کافی نته. به نظرم اینجاخدماتش بایه مسافرخونه فرق چندانی نداره فقط ظاهرشوتغییردادن
   Tokens: بدترین تجربه اقامت درهتل به اصطلاح ۵ستاره را داشتم به نظرم اگه میخوایدپولتونوبریزید دوربریدبه هتلی مثل هتل داریوش که از بدو ورود ضعف مدیریت درتمام قسمتهاش به چشم میخوره . اطلاعات سایتشون که دروغه مثلا نوشته شده استخرجهت بانوان که سالهاست راه اندازی نشده یاسرویس کافی شا

In [ ]:
encoding = tokenizer.encode_plus(
    sample_comment,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2, 10422,  4246,  6032,  7081,  4061,  2789,  5831,  1459, 40729,
          2803,  8065,  2789, 12269, 12516, 68563, 11268,  4957, 15946, 10928,
          3021, 12296,  2954, 22565,  3826,  5335,  8985,  2800,  2791, 12382,
          4247,     4]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
class sentimentDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for sentiment analysis """

    def __init__(self, tokenizer, comments, targets=None, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = sentimentDataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
label_list = ['negative', 'neural','positive']
train_data_loader = create_data_loader(train['comment'].to_numpy(), train['label'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['comment'].to_numpy(), valid['label'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['comment'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['comment'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'][0])

dict_keys(['comment', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['موقعیت عالی برخورد مناسب دسترسی و منظره عالی نظافت خوب. اینترنت بسیار بد و ضعیف که خیلی از مواقع امکان ارتباط فراهم نیست عدم وجود اسانسور در بعضی از اطاقها برای کسانی که مشکل جسمی دارند قابل توجه هست. در حد یک هتل سه ستاره بسیار خوب هست و در کل برای چند شب خاطره خوبی خواهید داشت', 'خوب بود نسبتا. فقط ساعت تحویل اتاق خیلی بعد هست چون بیستر پروازها بعد طهر می\u200cباشد یعنی قبل اینکه ناهار بخوری وسایلت باید بیرون باشه و چند ساعتی توی گرمای کیش بیرون باشی و خیلی بعده شاید هم بیشتر هتلها این قانون باشه در کل خیلی سخته..', 'هتل بسیار تمیز و نسبتآ شیکی است اتاق\u200cها بسیار تمیز و لابی شیک و قشنگی دارد. برخورد پرسنل معمولی بود و اتاق\u200cها نورگیر نبود. صبحانه خوب بود.', 'سلام من ۱۷ بهمن به مدت ۵روز در این هتل اقامت داشتم. فضای باغ بسیار زیبا.. لابی هتل زیبا …چون کلا آدم غذا خوری نیستم در مورد کیفیت کلی غذاها نظر ندارم ولی بدک نبود.. ولی از نظر کلی وسایل اتاقها خیلی خیلی کهنه و قدیمی بود و از نظر بهداشت خی

In [ ]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['comment', 'input_ids', 'attention_mask', 'token_type_ids'])


In [ ]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids , return_dict=False)
        
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits 

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [ ]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.SentimentModel'>


In [ ]:
# sample data output

sample_data_comment = sample_data['comment']
sample_data_input_ids = sample_data['input_ids']
sample_data_attention_mask = sample_data['attention_mask']
sample_data_token_type_ids = sample_data['token_type_ids']
sample_data_targets = sample_data['targets']

# available for using in GPU
sample_data_input_ids = sample_data_input_ids.to(device)
sample_data_attention_mask = sample_data_attention_mask.to(device)
sample_data_token_type_ids = sample_data_token_type_ids.to(device)
sample_data_targets = sample_data_targets.to(device)


# outputs = F.softmax(
#     pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids), 
#     dim=1)

outputs = pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids)
_, preds = torch.max(outputs, dim=1)

print(outputs[:5, :])
print(preds[:5])

tensor([[ 0.3675,  0.0115, -0.4235],
        [ 0.1884, -0.1241,  0.0005],
        [ 0.1827,  0.0932, -0.0717],
        [ 0.2763, -0.4311, -0.3728],
        [ 0.2137,  0.0096, -0.3568]], device='cuda:0',
       grad_fn=<SliceBackward0>)
tensor([0, 0, 0, 0, 0], device='cuda:0')


In [ ]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [ ]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/25 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/25 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/25 [00:00<?, ?it/s]

Evaluation... :   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_comments = test['comment'].to_numpy()
preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/2 [00:00<?, ?it/s]

(49,) (49, 3)


In [ ]:
y_test, y_pred = [label_list.index(label) for label in test['label'].values], preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.774579305406373

              precision    recall  f1-score   support

    negative       0.86      0.75      0.80        16
      neural       0.71      0.88      0.79        17
    positive       0.79      0.69      0.73        16

    accuracy                           0.78        49
   macro avg       0.79      0.77      0.77        49
weighted avg       0.78      0.78      0.77        49

